In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName("News Flash").getOrCreate()

In [2]:
news_flash = spark.read.text('market-basket.csv')

In [34]:
news_flash.take(1)[0].value

'BillNo;Itemname;Quantity;Date;Price;CustomerID;Country'

In [4]:
sliced  = news_flash.rdd.map(lambda i: i.split(";")).map(lambda i: (i[0], i[1], i[2], i[3], i[4], i[5], i[6]))

In [6]:
train, valid, test = sliced.randomSplit([6, 2, 2], seed = 99)

In [9]:
validing = valid.map(lambda i: (i[0], i[1], i[2], i[3], i[5], i[6]))

In [10]:
predict = test.map(lambda i: (i[0], i[1], i[2], i[3], i[5], i[6]))

In [11]:
from pyspark.mllib.recommendation import ALS
import math

seed = 99
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

In [13]:
for rank in ranks:
    model = ALS.train(train, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print('For rank',rank,'the RMSE is',error)
    if error < min_error:
        min_error = error
        best_rank = rank

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (10.21.92.160 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.schedul